# APRiL 
### Advanced Passive Radar Library
Target Localization Module <br>

Algoritm implementations in this module solves the multistatic localization problem according to the description in [1].

In order to calculate the descartes coordinates of a target we need the descartes coordinates of the radar, the used illuminators and the measured bistatic ranges as apriori knowledge. In [1] two different algorithms are described to solve the problem. The Spherical  Interpolation (SI) and the Spherical Intersection (SX) methods. Currently pyAPRiL only implements the later in the target localization module.


## 1. Multistatic Localization
### 1.1 <span style="color:Olive"> localize_target_ms_sx: </span>

*rd_matrix = <span style="color:DarkKhaki">localize_target_ms_sx</span>(ioo_coords, rb_vec)* 

This function solves the multistatic localization problem using the Spherical intersection approcch described in [1]
    
**Parameters:**
- *ioo_coords*: <span style="color:RoyalBlue">float numpy array with size (NTx x 3), where NTx is the 
        number of illuminators </span>
> x,y,z descartes coordinates of the used illuminators [m]
- *rb_vec*: <span style="color:RoyalBlue">float numpy array with size (NTx x 1), where NTx is the 
        number of illuminatorss </span>
> measured bistatic ranges [m]
        
**Returns:**
- *x1,x2* <span style="color:RoyalBlue"> float numpy arrays (1 x 3) </span> 
> 2 Solutions of the calculated x,y,z descartes coordinates of the target. 

The following code section shows the utilization of the function.

In [1]:
import numpy as np
from pyapril.targetLocalization import localize_target_ms_sx

def calculate_bistatic_range(ioo_coords, radar_coords, target_coords):
    # Baseline distance
    L = np.sqrt(np.sum(np.abs(radar_coords-ioo_coords)**2)) 
    
     # Target to IoO distance
    Rt = np.sqrt(np.sum(np.abs(target_coords-ioo_coords)**2)) 
    
    # Target to radar distance
    Rr = np.sqrt(np.sum(np.abs(target_coords-radar_coords)**2)) 
    
    # Bistatic distance
    Rb = Rt+Rr-L

    return Rb

"""
PARAMETERS
"""
ioo_coords = np.array([[0.0, -100.0, -500.0],
                      [500.0, 500.0, 2000.0],
                      [-2000.0, 2000.0, 0.0]])


target_coords = np.array([0.0, 1000.0, 0.0])

"""
TEST
"""

rb_vec =  np.array([calculate_bistatic_range(ioo_coords[i,:], np.array([0,0,0]), target_coords) for i in range(ioo_coords.shape[0])])
x1,x2 = localize_target_ms_sx(ioo_coords, rb_vec)

print("Target coordinate, solution #1", x1)
print("Target coordinate, solution #2", x2)

# Calculate errors
err = np.max(abs(np.array([x1,x2]) -target_coords), axis=1)

print("Maximum coordinate error:",err)



Target coordinate, solution #1 [-5.34896571e-11  1.00000000e+03 -1.30739863e-12]
Target coordinate, solution #2 [-872.82156541   55.32462363  385.06828654]
Maximum coordinate error: [5.34896571e-11 9.44675376e+02]


### Utlization Notes
The following points give some guidance on the real world application of the algorithm.
- The used illuminators should not share an equal coordinate value, otherwise the  calculation fail. (Singular matrix) E.g.: multiple illuminators have the same height value
- In order to use this algorithm one has to known the correct pairing of the measured bistatic range values and the illuminators. This could be difficult when using DVB-T SFN illuminators, and hence auxiliary processing steps are required to determine it.
- The calculation always results in two possible coordinates. It is task of the upcoming processing steps to decide which one is valid. One can make this decision based on direction of arrival information about the target  (if exist), or by comparing the altitudes of the calculated target positions.


### References
[1] Mateusz Malanowski: *Signal Processing for Passive Bistatic Radar*, 2019 ARTECH HOUSE 685 Canton Street Norwood, MA 02062, Section 8.2.1

### Further Work
**Multistatic localization**
- [F-MS1] [1] Describes an alternative localization method, called Spherical Interpolation, which is claimed to be less accurate in Passive Radar use cases, but also used when it comes to solving TDOA problems.

#### Authors
dr. Tamas Peto, PhD <br>
Initial version: 2023 10 01